In [900]:
"""
!pip3 install folium
!pip3 install wget
!pip install folium==0.8.3
"""

import folium
import wget
import pandas as pd

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [901]:
# Download and read the `spacex_launch_geo.csv`
# spacex_csv_file = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/
#   IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv")

#Read Data from CSV
spacex_df = pd.read_csv("spacex_launch_geo.csv")

In [902]:
# Select relevant sub-columns, remove unnecessary columns
spacex_df = spacex_df[["Launch Site", "Lat", "Long", "class"]]

# Combine CCAFS LC-40 and CCAFS SLC-40 into CCAFS, rename KSC LC-39A to KSC, and VAFB SLC-4E to VAFB
spacex_df["Launch Site"] = spacex_df["Launch Site"].replace(
    {"CCAFS LC-40": "CCAFS", "CCAFS SLC-40": "CCAFS", "KSC LC-39A": "KSC", "VAFB SLC-4E":"VAFB"})

# Fix KSC Launch site lat and long 
spacex_df.loc[spacex_df["Launch Site"] == "KSC", "Lat"] = 28.608389
spacex_df.loc[spacex_df["Launch Site"] == "KSC", "Long"] = -80.604333

#group the df by launch site and rename columns
Launch_Sites_df = spacex_df.groupby(["Launch Site"], as_index=False).first()

# Add Long form Launch Site Name
Launch_Sites_df["Launch_Site_Long_Form"] = ["Cape Canaveral Space Launch Complex",
                                        "Kennedy Space Center Launch Complex",
                                        "Vandenberg Space Launch Complex"]

# rename some columns from Launch_Sites_df
cols_to_rename = {"Lat": "Launch_Site_Lat", "Long": "Launch_Site_Long", "Launch Site":"Launch_Site_Abbreviation"}
Launch_Sites_df = Launch_Sites_df.rename(columns=cols_to_rename)

#Drop "class" column from Launch_Sites_df
Launch_Sites_df = Launch_Sites_df.drop(columns=["class"])

#add empty columns to Launch_Sites_df
new_columns = ["Closest_Coastline", "Closest_City", "Closest_Railway", "Closest_Highway", "Launch_Site_Lat",
                "Launch_Site_Long", "Coastline_Lat", "Coastline_Long", "City_Lat", "City_Lat", "City_Long",
                "Railway_Lat", "Railway_Long", "Highway_Lat", "Highway_Long"]
Launch_Sites_df[new_columns] = None

#Launch_Sites_df

In [903]:
# Define a dictionary to store the coordinates for each location type
coordinates = {
    "Launch_Site": {"Lat": [28.562302, 28.608389, 34.632834], "Long": [-80.577356, -80.604333, -120.610745]},
    "Coastline": {"Lat": [28.576994, 28.63994, 34.63149], "Long": [-80.51157, -80.54866, -120.69957]},
    "City": {"Lat": [28.40585, 28.60623, 34.646111], "Long": [-80.58986, -80.80942, -120.460278]},
    "Railway": {"Lat": [28.57211, 28.6118, 34.62179], "Long": [-80.58529, -80.5981, -120.63092]},
    "Highway": {"Lat": [28.359850, 28.501658, 34.713189], "Long": [-80.794684, -80.838687, -120.489079]}
}
# Iterate over the dictionary and add the latitude and longitude columns to the dataframe
for key, value in coordinates.items():
    Launch_Sites_df[f"{key}_Lat"] = value["Lat"]
    Launch_Sites_df[f"{key}_Long"] = value["Long"]

In [904]:
# Start location is NASA Johnson Space Center
Start_location = [29.559684888503615, -95.0830971930759] # NASA Johnson Space Center

# Initialize the map
site_map = folium.Map(location=Start_location, zoom_start = 4)

In [905]:
# Create a blue circle at NASA Johnson Space Center"s coordinate with a popup label showing its name
coordinate = [29.559684888503615, -95.0830971930759] # NASA Johnson Space Center
circle = folium.Circle(coordinate, radius=1000, color="#0000ff",fill=True).add_child(folium.Popup("NASA Johnson Space Center"))
site_map.add_child(circle);

# Create a blue circle at NASA Johnson Space Center"s coordinate with a icon showing its name
marker = folium.map.Marker(
    coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        #rotation = 45,
        icon_size=(300, 20),# width, height
        icon_anchor=(-10, 0),# +Left, +up
        html="<div style=\"font-size: 12; color: cadetblue;\"><b>%s</b></div>" % "NASA Johnson Space Center",
    )
)
site_map.add_child(marker);

#site_map

In [906]:
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values.
#   In addition, add Launch site name as a popup label

#for i, row in Launch_Sites_df.iterrows():
for i in range(0, len(Launch_Sites_df)):
    coordinate = [Launch_Sites_df.loc[i, "Launch_Site_Lat"], Launch_Sites_df.loc[i, "Launch_Site_Long"]]
    
    circle = folium.Circle(coordinate, radius=1000, color="#0000ff",fill=True).add_child(folium.Popup(Launch_Sites_df.loc[i,"Launch_Site_Long_Form"]))
    site_map.add_child(circle);
    
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            #rotation = 45,
            icon_size=(300, 20), # width, height
            icon_anchor=(-10, 0), # +Left, +up
        html="<div style=\"font-size: 12; color:cadetblue;\"><b>%s</b></div>" % Launch_Sites_df.loc[i,"Launch_Site_Long_Form"]))
    site_map.add_child(marker);
    
#site_map

In [907]:
# Create MarkerCluster Object
marker_cluster = MarkerCluster()

In [908]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return "green"
    else:
        return "red"

spacex_df["marker_color"] = spacex_df["class"].apply(assign_marker_color)

In [909]:
# Create and add a Marker cluster to the site map
for index, row in spacex_df.iterrows():  
    coordinate = [row["Lat"], row["Long"]]
    folium.map.Marker(coordinate, icon=folium.Icon(
        color="white", icon_color=row["marker_color"])).add_to(marker_cluster) #lightblue

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster);

#site_map

In [910]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position="topright",
    separator=" Long: ",
    empty_string="NaN",
    lng_first=False,
    num_digits=20,
    prefix="Lat:",
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position);

#site_map

In [911]:
# Function to calculate distance
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [912]:
# Create a marker (in km) with distance to a closest coastline, city, railway, and highway.
def calculate_distance_and_create_marker(row, location_type, color):
    temp_distance = 0
    temp_distance = calculate_distance(row["Launch_Site_Lat"], row["Launch_Site_Long"],
                                        row[location_type + "_Lat"], row[location_type + "_Long"])
    row["Closest_" + location_type] = temp_distance

    distance_marker = folium.Marker(
        [row[location_type + "_Lat"], row[location_type + "_Long"]],
        icon=DivIcon(
            icon_size=(100, 10), # width, height
            icon_anchor=(0, 0), # +Left, +up
            html="<div style=\"font-size: 12; color:{};\"><b>{:10.2f} km</b></div>".format(color, temp_distance),
        )
    )
    site_map.add_child(distance_marker)
    return row

location_types = ["Coastline", "City", "Railway", "Highway"]
colors = ["blue", "red", "green", "purple"]

for location_type, color in zip(location_types, colors):
    Launch_Sites_df = Launch_Sites_df.apply(calculate_distance_and_create_marker, location_type=location_type, color=color, axis=1)

#Launch_Sites_df

In [913]:
# Create Polyline to a closest coastline, city, railway, and highway.

def create_polyline(row, location_type, color):
    coordinates = [[row["Launch_Site_Lat"], row["Launch_Site_Long"]],
                    [row[location_type + "_Lat"], row[location_type + "_Long"]]]
    line = folium.PolyLine(locations=coordinates, weight=2, color=color)
    site_map.add_child(line)

location_types = ["Coastline", "City", "Railway", "Highway"]
colors = ["blue", "red", "green", "purple"]

for location_type, color in zip(location_types, colors):
    Launch_Sites_df.apply(create_polyline, location_type=location_type, color=color, axis=1)

In [914]:
# Save Data Map to File
site_map.save("Folium Map of Launch Cites With Proximity Lines.html")

Launch_Sites_df.to_csv("Launch_Sites_df", index=False)
spacex_df.to_csv("spacex_df_after", index=False)

In [915]:
site_map
#Launch_Sites_df
#spacex_df